In [ ]:
[
 HighBP, HighChol, HeartDiseaseorAttack, Stroke,
 Smoker, PhysActivity, DiffWalk,
 BMI, MentHlth, PhysHlth, GenHlth,
 Sex, Age, HvyAlcoholConsump,
]




Age,Sex,BMI,HighBP,PhysActivity, Smoker, HighChol,HeartDiseaseorAttack,Stroke,DiffWalk,MentHlth,PhysHlth,GenHlth,HvyAlcoholConsump

In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.calibration import CalibratedClassifierCV

In [32]:
data=pd.read_csv("cdc_diabetes_health_indicators.csv")

In [33]:
data.head()

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,...,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,Diabetes_binary
0,1,1,1,40,1,0,0,0,0,1,...,0,5,18,15,1,0,9,4,3,0
1,0,0,0,25,1,0,0,1,0,0,...,1,3,0,0,0,0,7,6,1,0
2,1,1,1,28,0,0,0,0,1,0,...,1,5,30,30,1,0,9,4,8,0
3,1,0,1,27,0,0,0,1,1,1,...,0,2,0,0,0,0,11,3,6,0
4,1,1,1,24,0,0,0,1,1,1,...,0,2,3,0,0,0,11,5,4,0


In [34]:
features = [
    "Diabetes_binary","HighBP", "HighChol", "HeartDiseaseorAttack", "Stroke",
    "Smoker", "PhysActivity", "DiffWalk",
    "BMI", "MentHlth", "PhysHlth", "GenHlth",
    "Sex", "Age", "HvyAlcoholConsump",
]

In [35]:
df = data[features]

df = df.dropna()

x = df.drop('Diabetes_binary', axis=1)
y = df['Diabetes_binary']

In [36]:
x.head()

,HighBP,HighChol,HeartDiseaseorAttack,Stroke,Smoker,PhysActivity,DiffWalk,BMI,MentHlth,PhysHlth,GenHlth,Sex,Age,HvyAlcoholConsump
0,1,1,0,0,1,0,1,40,18,15,5,0,9,0
1,0,0,0,0,1,1,0,25,0,0,3,0,7,0
2,1,1,0,0,0,0,1,28,30,30,5,0,9,0
3,1,0,0,0,0,1,0,27,0,0,2,0,11,0
4,1,1,0,0,0,1,0,24,3,0,2,0,11,0


In [37]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, stratify=y, random_state=42)

In [38]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [39]:
smote = SMOTE(random_state=42)
x_train_resampled, y_train_resampled = smote.fit_resample(x_train_scaled, y_train)

In [40]:
print("Before SMOTE:", y_train.value_counts())
print("After SMOTE:", y_train_resampled.value_counts())

Before SMOTE: Diabetes_binary
0    152834
1     24742
Name: count, dtype: int64
After SMOTE: Diabetes_binary
0    152834
1    152834
Name: count, dtype: int64


In [42]:
x_train

,HighBP,HighChol,HeartDiseaseorAttack,Stroke,Smoker,PhysActivity,DiffWalk,BMI,MentHlth,PhysHlth,GenHlth,Sex,Age,HvyAlcoholConsump
231121,0,1,0,0,1,1,0,22,0,0,2,0,12,0
145098,0,1,0,0,1,1,1,23,0,3,2,0,7,1
101727,0,0,0,0,0,1,0,27,0,0,1,1,8,1
229088,1,0,0,0,1,1,0,28,7,0,2,0,3,0
228686,0,0,0,0,1,1,0,26,0,0,1,0,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134394,0,1,0,0,1,0,0,29,0,1,3,0,9,0
209330,0,0,0,0,0,0,0,26,0,25,2,0,3,0
179576,0,1,0,0,0,1,0,26,10,25,3,0,13,0
246770,1,1,1,1,1,1,0,35,0,10,3,1,10,0


In [43]:


base_svc = LinearSVC(random_state=42, max_iter=10000)
model = CalibratedClassifierCV(base_svc)  # Adds probability support
model.fit(x_train_resampled, y_train_resampled)


CalibratedClassifierCV(estimator=LinearSVC(max_iter=10000, random_state=42))

In [44]:
def predict_diabetes(model, scaler, HighBP, HighChol, HeartDiseaseorAttack, Stroke, Smoker, PhysActivity, DiffWalk, BMI, MentHlth, PhysHlth, GenHlth, Sex, Age, HvyAlcoholConsump):
    new_patient = [[Age,Sex,BMI,HighBP,PhysActivity, Smoker, HighChol,HeartDiseaseorAttack,Stroke,DiffWalk,MentHlth,PhysHlth,GenHlth,HvyAlcoholConsump]]
    new_patient_scaled = scaler.transform(new_patient)
    prediction = model.predict(new_patient_scaled)
    
    if prediction[0] == 1:
        print("⚠️  The patient is likely diabetic.")
    else:
        print("✅  The patient is likely not diabetic.")

In [45]:
predict_diabetes(
    model=model,
    scaler=scaler,
    HighBP=1,
    HighChol=1,
    HeartDiseaseorAttack=1,
    Stroke=1,
    Smoker=1,
    PhysActivity=1,
    DiffWalk=1,
    BMI=34, 
    MentHlth=0,
    PhysHlth=7,
    GenHlth=4,
    Sex=0,
    Age=9,
    HvyAlcoholConsump=0,
)

⚠️  The patient is likely diabetic.


C:\Users\slowd\OneDrive\Desktop\Projects\MLProjects\Diabeties\diabeties\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [46]:
import pandas as pd
from sklearn.utils import resample

# If x_train is a NumPy array, convert it to DataFrame
# Provide feature names if needed
feature_names = [
    "HighBP", "HighChol", "HeartDiseaseorAttack", "Stroke",
    "Smoker", "PhysActivity", "DiffWalk",
    "BMI", "MentHlth", "PhysHlth", "GenHlth",
    "Sex", "Age", "HvyAlcoholConsump"
]
x_train_df = pd.DataFrame(x_train, columns=feature_names)

# Convert y_train to Series
y_train_series = pd.Series(y_train, name="Diabetes_binary")

# Combine X and y into one DataFrame
train_data = pd.concat([x_train_df, y_train_series], axis=1)

# Separate majority and minority classes
df_majority = train_data[train_data['Diabetes_binary'] == 0]
df_minority = train_data[train_data['Diabetes_binary'] == 1]

In [47]:
df_majority_downsampled = resample(
    df_majority,
    replace=False,
    n_samples=len(df_minority),
    random_state=42
)

# Combine for balanced data
df_balanced = pd.concat([df_majority_downsampled, df_minority])

# Shuffle and reset index
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Final X and y for SHAP
x_shap = df_balanced.drop(columns=['Diabetes_binary'])
y_shap = df_balanced['Diabetes_binary']

In [48]:
x_shap.head()

,HighBP,HighChol,HeartDiseaseorAttack,Stroke,Smoker,PhysActivity,DiffWalk,BMI,MentHlth,PhysHlth,GenHlth,Sex,Age,HvyAlcoholConsump
0,1,1,0,0,0,0,0,33,0,3,3,0,9,0
1,1,1,0,0,0,0,1,40,0,9,3,0,11,0
2,1,1,0,0,0,1,1,33,15,10,4,1,8,0
3,0,1,0,0,0,1,0,21,0,30,5,1,6,0
4,1,1,0,0,0,0,1,39,0,30,4,0,13,0


In [49]:
y_shap.head()

0    1
1    1
2    1
3    1
4    0
Name: Diabetes_binary, dtype: int64

In [50]:
import joblib

# Save the model
joblib.dump(model, 'final_model.pkl')

# Save the scaler
joblib.dump(scaler, 'scaler.pkl')

# Save the balanced SHAP input data
x_shap.to_csv('x_shap.csv', index=False)
y_shap.to_csv('y_shap.csv', index=False)


In [17]:
predict_diabetes(
    model=model,
    scaler=scaler,
    HighBP=0,
    HighChol=0,
    HeartDiseaseorAttack=0,
    Stroke=0,
    Smoker=1,
    PhysActivity=1,
    DiffWalk=0,
    BMI=25, 
    MentHlth=0,
    PhysHlth=0,
    GenHlth=3,
    Sex=0,
    Age=7,
    HvyAlcoholConsump=0,
)

✅  The patient is likely not diabetic.


C:\Users\slowd\OneDrive\Desktop\Projects\MLProjects\Diabeties\diabeties\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [51]:
base_svc = LinearSVC(random_state=42, max_iter=10000)

# Step 2: Wrap it with CalibratedClassifierCV to enable predict_proba
model = CalibratedClassifierCV(base_svc)

# Step 3: Fit the model
model.fit(x_train_resampled, y_train_resampled)

# Step 4: Predict
y_pred = model.predict(x_test_scaled)

# Step 5: Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.7304083885209713
Confusion Matrix:
 [[47512 17988]
 [ 2529  8075]]
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.73      0.82     65500
           1       0.31      0.76      0.44     10604

    accuracy                           0.73     76104
   macro avg       0.63      0.74      0.63     76104
weighted avg       0.86      0.73      0.77     76104

